In [1]:
%matplotlib qt4
from __future__ import division

import math

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

import matplotlib as mpl

mpl.rcParams['text.usetex']=False
mpl.rcParams['text.latex.unicode']=False

In [2]:
data = tools.load_data(offset=1900000, limit=100000)
#data = data[filters.open_questions(data)]
#data = tools.unknown_answers(data)

Loaded 82810 answers.


In [3]:
class PFAGongWithLogging(models.PFAGong):
    
    def __init__(self, *args, **kwargs):
        super(PFAGongWithLogging, self).__init__(*args, **kwargs)
        self.timing = []
        
    def update(self, answer):
        item = self.items[answer.user_id, answer.place_id]
        if item.practices:
            diff = tools.time_diff(answer.inserted, item.last_inserted)
            self.timing += [(diff, answer.is_correct, self.predict(answer))]
        super(PFAGongWithLogging, self).update(answer)
        
class PFASpacingWithLogging(models.PFASpacing):
    
    def __init__(self, *args, **kwargs):
        super(PFASpacingWithLogging, self).__init__(*args, **kwargs)
        self.timing = []
        
    def update(self, answer):
        item = self.items[answer.user_id, answer.place_id]
        if item.practices:
            diff = tools.time_diff(answer.inserted, item.last_inserted)
            self.timing += [(diff, answer.is_correct, self.predict(answer))]
        super(PFASpacingWithLogging, self).update(answer)
        
        
class PFAForgettingWithLogging(models.PFAForgetting):
    
    def __init__(self, *args, **kwargs):
        super(PFAForgettingWithLogging, self).__init__(*args, **kwargs)
        self.timing = []
        
    def update(self, answer):
        item = self.items[answer.user_id, answer.place_id]
        if item.practices:
            diff = tools.time_diff(answer.inserted, item.last_inserted)
            self.timing += [(diff, answer.is_correct, self.predict(answer))]
        super(PFAForgettingWithLogging, self).update(answer)


class PFAStaircaseWithLogging(models.PFAStaircase):
    
    def __init__(self, *args, **kwargs):
        super(PFAStaircaseWithLogging, self).__init__(*args, **kwargs)
        self.timing = []
        
    def update(self, answer):
        item = self.items[answer.user_id, answer.place_id]
        if item.practices:
            diff = tools.time_diff(answer.inserted, item.last_inserted)
            self.timing += [(diff, answer.is_correct, self.predict(answer))]
        super(PFAStaircaseWithLogging, self).update(answer)

In [4]:
def time_effect_log(t):
    return 1.6 - 0.1*np.log(t)
def time_effect_d80(t):
    return 80 / t if t > 0 else 100

In [9]:
def chunks(l, n):
    for i in xrange(0, len(l), n):
        yield l[i:i+n]

def interval_error(timings, interval_size=500, metric=tools.rmse):

    answers = sorted(timings, key=lambda p: p[0])

    def get_diffs_mean(chunk):
        return np.mean([diff for diff, _, _ in chunk])
    def get_answers_mean(chunk):
        return np.mean([pred - obs for _, obs, pred in chunk])
    def get_answers_value(chunk):
        predictions = [pred for _, obs, pred in chunk if np.isfinite(pred)]
        observation = [obs for _, obs, pred in chunk if np.isfinite(pred)]
        return metric(observation, predictions)

    return [
        (get_diffs_mean(chunk), get_answers_value(chunk))
        for chunk in chunks(answers, interval_size)
    ]

In [19]:
pfas = PFASpacingWithLogging(models.EloModel())
pfas.train(data)

In [6]:
pfaf = PFAForgettingWithLogging(models.EloModel(), time_effect_fun=lambda t: 1.2 - 0.08*np.log(t))
pfaf.train(data)

In [27]:
pfag = PFAGongWithLogging(models.EloModel())
pfag.train(data)

In [21]:
pfast = PFAStaircaseWithLogging(models.EloModel(),
    staircase={
        (0, 60): 1.5,
        (60, 90): 1.15,
        (90, 150): 0.95,
        (150, 300): 0.8,
        (300, 600): 0.7,
        (600, 1800): 0.7,
        (1800, 10800): 0.65,
        (10800, 86400): 0.6,
        (86400, 259200): 0.4,
        (259200, 2592000): 0.02,
        (2592000, np.inf): 0.0,
    }
)
pfast.train(data)

In [22]:
intervals1 = interval_error(pfas.timing, interval_size=9000)

In [10]:
intervals2 = interval_error(pfaf.timing, interval_size=9000)

In [24]:
intervals3 = interval_error(pfast.timing, interval_size=9000)

In [28]:
metric = lambda y_true, y_pred: (np.mean(y_true), np.mean(y_pred))
intervals1 = interval_error(pfaf.timing, interval_size=7000, metric=metric)
intervals2 = interval_error(pfast.timing, interval_size=7000, metric=metric)
intervals3 = interval_error(pfag.timing, interval_size=7000, metric=metric)
p1 = plt.plot([x[0] for x in intervals1], [x[1][1] - x[1][0] for x in intervals1], '.-', color='#59d700')
p2 = plt.plot([x[0] for x in intervals2], [x[1][1] - x[1][0] for x in intervals2], '.-', color='orange')
p3 = plt.plot([x[0] for x in intervals3], [x[1][1] - x[1][0] for x in intervals3], '.-')
plt.xscale('log')
plt.ylabel('observed - predicted')
plt.xlabel('time from previous attempt (seconds)')
plt.xlim([min([x[0] for x in intervals1]) - 20, max([x[0] for x in intervals1]) + 100000])
plt.legend([p1[0], p2[0], p3[0]], ('PFA Forgetting', 'PFA Staircase', 'PFA Gong'))
plt.show()

In [37]:
p1 = plt.plot([x[0] for x in intervals1], [x[1] for x in intervals1], '.-')
p2 = plt.plot([x[0] for x in intervals2], [x[1] for x in intervals2], '.-', color='#59d700')
p3 = plt.plot([x[0] for x in intervals3], [x[1] for x in intervals3], '.-', color='orange')
plt.xscale('log')
plt.ylabel('RMSE')
plt.xlabel('time from previous attempt (seconds)')
plt.xlim([min([x[0] for x in intervals2]) - 20, max([x[0] for x in intervals2]) + 100000])
plt.legend([p2[0], p1[0], p3[0]], ('PFA Gong + Forgetting', 'PFAE + Forgetting', 'PFAE + Staircase',))
plt.show()

In [15]:
pfaf1 = PFAForgettingWithLogging(models.EloModel(), time_effect_fun=lambda t: 1.2 - 0.08*np.log(t))
pfaf1.train(data)

In [6]:
pfaf2 = PFAForgettingWithLogging(models.EloModel(), time_effect_fun=lambda t: 1.4 - 0.1*np.log(t))
pfaf2.train(data)

In [64]:
pfaf3 = PFAForgettingWithLogging(models.EloModel(), time_effect_fun=time_effect_d80)
pfaf3.train(data)

In [16]:
intervals1 = interval_error(pfaf1.timing, interval_size=9000)
intervals2 = interval_error(pfaf2.timing, interval_size=9000)
intervals3 = interval_error(pfaf3.timing, interval_size=9000)

In [17]:
p1 = plt.plot([x[0] for x in intervals1], [x[1] for x in intervals1], '.-')
p2 = plt.plot([x[0] for x in intervals2], [x[1] for x in intervals2], '.-', color='#59d700')
p3 = plt.plot([x[0] for x in intervals3], [x[1] for x in intervals3], '.-', color='orange')
plt.xscale('log')
plt.ylabel('RMSE')
plt.xlabel('time from previous attempt (seconds)')
plt.xlim([min([x[0] for x in intervals1]) - 20, max([x[0] for x in intervals1]) + 100000])
plt.legend([p1[0], p2[0], p3[0]], (r'$1.2\ -\ 0.1\ \log({t})$',
                                   r'$1.4\ -\ 0.1\ \log({t})$',
                                   r'$1.6\ -\ 0.1\ \log({t})$'))
plt.show()

In [10]:
data = tools.add_spacing(data)

In [12]:
ranges = [0, 60, 90, 150, 300, 600, 1800, 10800, 86400, 259200, 2592000]
intervals = {i: None for i in zip(ranges, ranges[1:] + [np.inf])}

for interval in intervals.keys():
    lower, upper = interval
    data_slice = data[(data['spacing'] > lower) & (data['spacing'] < upper)].copy()
    
    if data_slice.empty:
        continue
        
    print interval, len(data_slice)
    
    pfat = models.PFATiming(models.EloModel(), time_effect_fun=lambda t: t/80)
    pfat_test = PerformanceTest(pfat, data_slice)
    pfat_test.run()
    
    intervals[interval] = pfat_test.results['train'].off
    
intervals = sorted([(np.mean(interval), value) for interval, value in intervals.items()], key=lambda x: x[0])

(600, 1800) 5403
(150, 300) 12653
(90, 150) 9562
(86400, 259200) 2470
(300, 600) 6758
(259200, 2592000) 355
(10800, 86400) 4430
(60, 90) 6124
(1800, 10800) 4449
(0, 60) 4970


In [19]:
ranges = [0, 60, 90, 150, 300, 600, 1800, 10800, 86400, 259200, 2592000]
intervals = {i: None for i in zip(ranges, ranges[1:] + [np.inf])}

for interval in intervals.keys():
    lower, upper = interval
    data_slice = data[(data['spacing'] > lower) & (data['spacing'] <= upper)]
    
    if len(data_slice) > 0:
        correct = len(data_slice[data_slice['is_correct'] == 1]) / len(data_slice)
        intervals[interval] = correct
        
intervals.pop((2592000, np.inf))

In [20]:
intervals = sorted([(np.mean(interval), value) for interval, value in intervals.items()], key=lambda x: x[0])
plt.plot([x[0] for x in intervals], [x[1] for x in intervals])
plt.xscale('log')
plt.show()

In [18]:
ind = np.arange(len(intervals))    # the x locations for the groups
width = 0.50       # the width of the bars: can also be len(x) sequence

correctness = [intervals[i] * 100 for i in sorted(intervals)]
incorrectness = [(1 - intervals[i]) * 100 for i in sorted(intervals)]

p1 = plt.bar(ind, correctness, width, color='#7FFF24')
p2 = plt.bar(ind, incorrectness, width, color='#ff512e', bottom=correctness)

plt.ylabel('%')
plt.xticks(ind+width/2., ('60 s', '90 s', '150 s', '5 m', '10 m',
                          '30 m', '3 h', '24 h', '3 d', '30 d'))
plt.yticks(np.arange(0, 101, 10))
plt.legend((p1[0], p2[0]), ('correct', 'incorrect'), loc=4)

plt.show()

In [6]:
items = {}

for _, row in data.iterrows():
    index = (row.user_id, row.place_id)
    answer = models.Answer(**row.to_dict())
    if index in items:
        items[index].append(answer)
    else:
        items[index] = [answer]

In [7]:
ranges = [0, 60, 90, 150, 300, 600, 1800, 10800, 86400, 259200, 2592000]
intervals = zip(ranges, ranges[1:] + [np.inf])

def get_interval(value, list_of_intervals):
    for lower, upper in list_of_intervals:
        if lower < value <= upper:
            return lower, upper

correct_before = {i: [] for i in intervals}
incorrect_before = {i: [] for i in intervals}

for index in items:
    answers = sorted(items[index], key=lambda x: x.inserted)
    for a1, a2 in zip(answers, answers[1:]):
        diff = tools.time_diff(a2.inserted, a1.inserted)
        interval = get_interval(diff, intervals)
        if interval is None:
            continue
        if a1.is_correct:
            correct_before[interval].append(a2.is_correct)
        else:
            incorrect_before[interval].append(a2.is_correct)        

In [9]:
correct_intervals = {i: np.mean(v) for i, v in correct_before.items()}
incorrect_intervals = {i: np.mean(v) for i, v in incorrect_before.items()}

In [17]:
ind = (np.arange(len(intervals)-1) -1.2) * 1.15   # the x locations for the groups
width = 0.4       # the width of the bars: can also be len(x) sequence

correctness = [correct_intervals[i] * 100 for i in sorted(intervals) if i != (2592000, np.inf)]
incorrectness = [incorrect_intervals[i] * 100 for i in sorted(intervals) if i != (2592000, np.inf)]

p1 = plt.bar(ind-0.25, correctness, width, color='#7FFF24')
p2 = plt.bar(ind+0.25, incorrectness, width, color='#ff512e')

plt.ylabel('%')
plt.xticks(ind+width/2., ('60 s', '90 s', '150 s', '5 m', '10 m',
                          '30 m', '3 h', '24 h', '3 d', '30 d'))
plt.yticks(np.arange(0, 101, 10))
plt.legend((p2[0], p1[0]), ('incorrect before', 'correct before'), loc=4)

plt.show()